# Implementation of the Polyphase filterbank

In [1]:
import numpy as np

# object containing 512 samples of audio to be fed to the Polyphase Filterbank
class analysisBuffer:
    def __init__(self,bufferVal=np.zeros(512)):
        assert (len(bufferVal)==512),"Input not length 512!"
        self.bufferVal = bufferVal
    
    # by definition, 32 samples are pushed into the buffer at a time
    def pushBlock(self,sampleBlock):
        assert (len(sampleBlock)==32),"Sample block length not 32!"
        self.bufferVal[32:] = self.bufferVal[:-32]
        self.bufferVal[0:32] = sampleBlock

# analysis filterbank for MPEG Audio subband coding
def polyphaseFilterbank(x):
    # x - analysisBuffer object
    assert (type(x)==analysisBuffer),"Input not an analysisBuffer object!"
    
    C = np.load('data/mpeg_analysis_window.npy') # analysis window defined by MPEG
    assert (len(C)==512),"Window length not 512!"
    
    M = calcAnalysisMatrixCoeff()
    
    subbandSamples = np.zeros(32)
    for n in range(32):
        for k in range(63):
            for m in range(7):
                subbandSamples[n] = subbandSamples[n] + (M[n,k] * (C[k+64*m]*x.bufferVal[k+64*m]))
        
    return subbandSamples

# part of the Polyphase Filterbank
def calcAnalysisMatrixCoeff():
    M = np.zeros([32,64])
    for n in range(32):
        for k in range(64):
            M[n,k] = np.cos((2*n+1)*(k-16)*np.pi/64)
            
    return M
     

Simple test

In [2]:
# test pushBlock in Buffer and Filterbank just with random values
myBuffer = analysisBuffer(np.random.rand(512))
myBuffer.pushBlock(np.random.rand(32))

myst = polyphaseFilterbank(myBuffer)
print(myst)

[ 0.49309184 -0.0369007   0.07789391 -0.00640293  0.06286997 -0.03829349
  0.0267488  -0.06724389  0.09187528  0.07489049  0.09266429 -0.04235956
  0.03666419  0.08748308 -0.04932748 -0.0504179  -0.03817849 -0.07523898
  0.00713884  0.07646516  0.0277775  -0.04227305  0.01914915  0.06813109
  0.02262795  0.01057898  0.01137009  0.05170431  0.04791764  0.01926949
  0.04461921  0.02746576]


In [3]:
# test with a sine tone at 10kHz, assuming roughly 48kHz sampling rate
# --> maximum value in filterbank output seen in 13th entry

myBuffer = analysisBuffer(np.sin(2*np.pi*10000*np.linspace(0,0.01,512)))
myst = polyphaseFilterbank(myBuffer)
print(myst)

[ 1.21202680e-02 -1.44946970e-02 -9.28328898e-03  1.63199509e-02
  6.09067813e-03 -1.75211143e-02 -2.66505598e-03  1.80719505e-02
 -8.64472328e-04 -1.79703665e-02  4.35845671e-03  1.74879278e-02
  9.78679207e-01 -1.50207993e-02  1.05408444e-02  1.31735898e-02
 -1.32306672e-02 -1.06348660e-02  1.53482354e-02  7.64692128e-03
 -1.68664303e-02 -4.34626024e-03  1.77265416e-02  8.73449724e-04
 -1.79064471e-02  2.63321283e-03  1.73947968e-02 -6.04335340e-03
 -1.62124859e-02  9.22353398e-03  1.44085065e-02 -1.20454323e-02]


# Loading audio files

In [4]:
# this works

import scipy.io.wavfile as wav
filename = 'data/audio/watermelonman_audio.wav'
sampleRate, x=wav.read(filename)

xLeft  = x[:,0]
xRight = x[:,1]


# import routine which outputs bytes objects

#import wave
#chunk = 1024  
#open a wav format music
#filename = 'data/audio/lull_audio.wav'
#filename = 'data/audio/cometogether_audio.wav'
#f = wave.open(filename)

#f.getnchannels()
#f.getframerate()
#f.getnframes()
#f.getparams()
#f.readframes(1)

#f.close()

# more sophisticated audio library, not installed

#import librosa
#x, sample_rate = librosa.load('data/audio/lull_audio.wav', mono=True)


Routine that takes blocks of audio and feeds it to the buffer, then groups subband samples into frames

In [5]:
# object containing a single output of the polyphase filterbank, i.e. one sample of each of the 32 subband filters. 
# this is equicalent to 32 samples of audio input
class subbandSample:
    def __init__(self,sample=np.zeros(32)):
        assert (len(sample)==32),"Input length not 32!"
        self.sample = sample


# object containing a frame of 12 or 36 subbandSample objects, must be initialized empty first
class subbandFrame:
    def __init__(self,layer=1):
        assert (layer==1 or layer==2 or layer==3),"Encoding layer type not 1, 2 or 3!"
        self.layer=layer
        if self.layer==1:
            self.nSamples = 12
        elif self.layer ==2 or self.layer==3:
            self.nSamples = 36
        
        self.frame = np.zeros([32,self.nSamples])
        
    def pushFrame(self,subbandSamples):
        assert (not not subbandSamples),"No entries in subbandSamples list!"
        for i in range(self.nSamples):
            self.frame[:,i]=(subbandSamples[i].sample)
        subbandSamples = subbandSamples[self.nSamples:]   
        return subbandSamples

In [7]:
# takes raw audio signal, runs it into the analysisBuffer and calculated the filterbank outputs subbandSamples
def feedCoder(x):
    # x - mono or stereo PCM audio signal as array
    
    nSamples,stereo=x.shape
    assert (stereo==1 or stereo==2),"Input not mono or stereo!"
    
    if stereo==2:
        x  = x[:,0] # eventually will need to be xLeft/xRight and will need a whole routine to handle both channels
        #xRight = x[:,1]
    
    # zero-pad to divisible of 32 samples
    modulo32=nSamples%32
    nPadding = 32-modulo32
    xHold = x
    x = np.zeros([nSamples])
    x[0:nSamples] = xHold
    nSamples = nSamples+nPadding
    nBlocks = int(nSamples/32)
    assert (nSamples%32==0),"Zero-padding mistake!"
        
    xBuffer = analysisBuffer()
    iBlock = 0
    iSample = 0
    subbandSamples = []
    
    while iSample+32<=3072: # eventually needs to be nSamples
        xBuffer.pushBlock(x[iSample:iSample+32])
        subbandSamples.append(subbandSample(polyphaseFilterbank(xBuffer)))
        iBlock  += 1
        iSample += 32
    
    return subbandSamples


In [8]:
# simple test for feedCoder function

subbandSamples = feedCoder(x)

# show that output of feedCoder is a numerical array
print(len(subbandSamples))
print(subbandSamples[0].sample)
print(type(subbandSamples[0].sample))
print(type(subbandSamples[0].sample[0]))

96
[-8.88642068e-05 -1.33691841e-05  4.21516614e-05  2.42727167e-05
 -1.51464161e-05 -1.37671088e-05  1.19517563e-05  1.38434161e-05
 -4.84021110e-06 -8.13793121e-06  3.95610221e-06  2.70926853e-06
 -1.05200287e-05 -7.39344663e-06  1.11605375e-05  1.33518139e-05
 -5.74219727e-06 -1.34625862e-05  2.60376257e-06  1.22134456e-05
 -2.84394883e-06 -1.43296224e-05  2.98860468e-06  2.29187713e-05
  7.09263306e-06 -3.05895766e-05 -4.06421983e-05 -9.94862424e-06
  2.10387180e-05  1.40961667e-05 -2.35049370e-05 -5.58231500e-05]
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [6]:
# simple test for subbandFrame object
mySubFrame = subbandFrame(layer=1) # initialize
subbandSamples = mySubFrame.pushFrame(subbandSamples) # need to return popped samples list
#mySubFrame.pushFrame(subbandSamples[0:12])

print(mySubFrame.frame.shape)

NameError: name 'subbandSamples' is not defined

# Implementation of Calculation of scale factors

In [9]:
# compare max abs values of each subband frame to scalefactor table and deduct fitting scale factor
def calcScaleFactors(subbandFrame):
    if subbandFrame.layer == 1:
        assert (subbandFrame.frame.shape==(32, 12)),"Wrong subbandFrame array dimensions!"
        subbandMaxVals = np.amax(np.abs(subbandFrame.frame),axis=1)
    else:
        print("Error! Layer II and III coding not implemented yet")
        raise SystemExit(0)
    
    scaleFactorTable = np.load('data/mpeg_scale_factors.npy') # scale factors defined by MPEG
    scaleFactorTable = np.flip(scaleFactorTable)
    assert (len(scaleFactorTable)==64),"Table length not 64!"
    
    scaleFactor = np.zeros(32)
    scaleFactorIndex = []
    for iCompare in range(32):
        scaleFactor[iCompare]=scaleFactorTable[np.argmax(scaleFactorTable>subbandMaxVals[iCompare])]
        scaleFactorIndex.append(63-np.argmax(scaleFactorTable>subbandMaxVals[iCompare]))
    
    return scaleFactor, scaleFactorIndex

# convert scale factor indices into binary representation for coding
def codeScaleFactor(scaleFactorIndex):
    assert (type(scaleFactorIndex[0]==int)),"Input not integer value!"
    
    codedScaleFactor = []
    for iBand in range(len(scaleFactorIndex)):
        codedScaleFactor.append(bin(scaleFactorIndex[iBand]))
    
    return codedScaleFactor

In [10]:
# simple test
print(calcScaleFactors(mySubFrame))

# simple test to show binary representation os scalefactors
scaleFactor, scaleFactorIndex = calcScaleFactors(mySubFrame)

codeScaleFactor(scaleFactorIndex)

(array([1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06,
       1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06,
       1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06,
       1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06,
       1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06,
       1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06,
       1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06,
       1.20155435e-06, 1.20155435e-06, 1.20155435e-06, 1.20155435e-06]), [62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62])


['0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110',
 '0b111110']

# Bit allocation

In [11]:
def assignBits(subbandFrame):
    
    # for now it's just a fixed 8 bits per subband sample,  allocation routine will be included later
    nBits = 8*np.ones(32)
    
    return nBits

In [12]:
nBitsSubband = assignBits(mySubFrame)

Quantizer

In [13]:
# multiply subband frame by scalefactors and quantize them with the number of bits
def quantizeSubbandFrame(subbandFrame,scaleFactor,nBitsSubband):
    
    transmitScalefactor = []
    transmitSubband = []
    transmitNSubbands = []
    
    for iBand in range(32):
        if nBitsSubband[iBand]>0:
            transmitNSubbands.append(iBand)
            transmitScalefactor.append(scaleFactor[iBand])
            
            normalizedBand = subbandFrame.frame[iBand,:]*scaleFactor[iBand]
            quantizedBand = subbandQuantizer(normalizedBand,nBitsSubband[iBand])
            
            transmitSubband.append(codeSubband(quantizedBand,nBitsSubband[iBand]))
            
            return transmitSubband


# Quantizer defined by MPEG
def subbandQuantizer(normalizedBand,nBits):
    
    nSteps = int(2**nBits-1)
    indBits = int(nBits-2)
    print(type(indBits))
    nSteps = np.load('data/mpeg_qc_layer_i_nSteps.npy')
    A = np.load('data/mpeg_qc_layer_i_A.npy')
    B = np.load('data/mpeg_qc_layer_i_B.npy')
    
    quantizedBand = A[indBits]*normalizedBand+B[indBits]

    return quantizedBand

# convert scale factor indices into binary representation for coding
def codeSubband(quantizedBand,nBits):
    assert (type(scaleFactorIndex[0]==int)),"Input not integer value!"
    
    codedSubband = []
    for iSample in range(len(quantizedBand)):
        print(quantizedBand[iSample])
        codedVal    = bin(quantizedBand[iSample]) # this should be an integer number but isn't???
        codedVal    = codedVal[0:nBits]
        codedVal[0] = ~codedVal[0]
        codedSubband.append()
    
    return codedScaleFactor

In [14]:
quantizeSubbandFrame(mySubFrame,scaleFactor,nBitsSubband)

<class 'int'>
-0.00390625


TypeError: 'numpy.float64' object cannot be interpreted as an integer

# Miscellaneous

Convert right-left stereo array to mid-side coded array

In [15]:
def convRLtoMS(RLsig):
    
    nSamples,stereo=RLsig.shape
    assert stereo==2,"Not a stereo file!"
    MSsig = np.zeros([nSamples,2])
    
    MSsig[:,0] = (RLsig[:,0] + RLsig[:,1])/np.sqrt(2)
    MSsig[:,1] = (RLsig[:,0] - RLsig[:,1])/np.sqrt(2)
    
    return MSsig

In [16]:
# simple test

msx = convRLtoMS(x)

print(x[1,:])
print(msx[1,:])

[2 8]
[ 7.07106781 -4.24264069]
